# **Análise de Sentimento:**

 o processo de identificação e categorização computacional das opiniões expressas em um pedaço de texto, especialmente com o objetivo de determinar se a atitude do escritor em relação a um tópico específico, produto, etc., é positiva, negativa ou neutra.



In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"
!chmod 600 /content/kaggle.json


!kaggle datasets download -d crowdflower/first-gop-debate-twitter-sentiment
!unzip /content/first-gop-debate-twitter-sentiment -d /content/kaggle/

In [ ]:


import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
import re


Matendo apenas a colunas necessárias

In [ ]:
data = pd.read_csv('/content/kaggle/Sentiment.csv')
data = data[['text','sentiment']]

A seguir, estou eliminando os sentimentos 'Neutros', pois meu objetivo era apenas diferenciar tweets positivos e negativos. Depois disso, estou filtrando os tweets para que apenas textos e palavras válidas permaneçam. Em seguida, defino o número máximo de recursos como 2000 e uso o Tokenizer para vetorizar e converter o texto em Sequências para que a Rede possa lidar com ele como entrada.

In [ ]:
data = data[data.sentiment != "Neutral"]
data['text'] = data['text'].apply(lambda x: x.lower())
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

print(data[ data['sentiment'] == 'Positive'].size)
print(data[ data['sentiment'] == 'Negative'].size)

for idx,row in data.iterrows():
    row[0] = row[0].replace('rt',' ')

max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['text'].values)
X = tokenizer.texts_to_sequences(data['text'].values)
X = pad_sequences(X)

A seguir, eu configuro a Rede LSTM. Observe que as variáveis **embed_dim**, **lstm_out**, **batch_size** e **dropout_x** são hiperparâmetros; seus valores são de certa forma intuitivos e podem e devem ser ajustados para obter bons resultados. Por favor, observe também que estou usando a função de ativação softmax. A razão para isso é que nossa Rede está utilizando a entropia cruzada categórica (categorical crossentropy), e softmax é o método de ativação adequado para isso.

 Cria uma rede neural para tarefas de classificação de texto usando uma camada de incorporação seguida por uma camada LSTM e uma camada de saída softmax. O modelo pode ser treinado com dados de texto rotulados para realizar tarefas de classificação, como análise de sentimento.

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

embed_dim = 128  # Numero de dimensões para a camada de incorporação
lstm_out = 196  # número de unidades (neurônios) na camada LSTM

model = Sequential()
model.add(Embedding(max_fatures, embed_dim, input_length=X.shape[1]))
model.add(Dropout(0.2))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


Dataset para treino e teste

In [ ]:
Y = pd.get_dummies(data['sentiment']).values
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

Treinamento

In [ ]:
batch_size = 32
model.fit(X_train, Y_train, epochs = 25, batch_size=batch_size, verbose = 1)

Validação do treinamento

In [ ]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

Finalmente, estamos medindo o número de suposições corretas. Está claro que a Rede se sai muito bem ao identificar tweets negativos, mas não é tão eficaz em decidir se um tweet é positivo. Minha suposição educada aqui é que o conjunto de treinamento de tweets positivos é dramaticamente menor do que o de tweets negativos, o que explica os resultados "ruins" para os tweets positivos.

In [ ]:
pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
for x in range(len(X_validate)):

    result = model.predict(X_validate[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]

    if np.argmax(result) == np.argmax(Y_validate[x]):
        if np.argmax(Y_validate[x]) == 0:
            neg_correct += 1
        else:
            pos_correct += 1

    if np.argmax(Y_validate[x]) == 0:
        neg_cnt += 1
    else:
        pos_cnt += 1



print("pos_acc", pos_correct/pos_cnt*100, "%")
print("neg_acc", neg_correct/neg_cnt*100, "%")

In [ ]:
# Save the model to a file
model.save("sentiment_model_2.h5", save_format='tf')


In [ ]:
from keras.models import load_model

# Load do modelo
loaded_model = load_model("sentiment_model.h5")

# Agora você pode usar o load_model para fazer previsões sobre novos dados
# Por exemplo, você pode usar o tokenizer para pré-processar novos dados de texto e depois usar carregado_model.predict()

new_text = [" Have a excellent day", "This is a good review", "I didn't like this movie", 'Meetings: Because none of us is as dumb as all of us.', "I love Domino's pizza"]
new_text = [text.lower() for text in new_text]
new_text = [re.sub('[^a-zA-z0-9\s]', '', text) for text in new_text]

# Tokenize and pad the new text data
new_sequences = tokenizer.texts_to_sequences(new_text)
new_sequences = pad_sequences(new_sequences, maxlen=X.shape[1])

# Make predictions
predictions = loaded_model.predict(new_sequences)

# You can interpret the predictions based on your label encoding
for prediction in predictions:
    if np.argmax(prediction) == 0:
        print("Negative")
    else:
        print("Positive")
